<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M6_MLflow_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://ploomber.io/images/blog/mlflow2sql/header.png)

MLflow supports several tracking backend stores, including file systems, databases, object storage, and more. One of the supported database backends is SQLite. SQLite is a file-based database system that is lightweight and easy to use. It is also included with most operating systems, making it a popular choice for local development and testing.

Using SQLite as the tracking backend for MLflow has several benefits, including:

> It requires minimal setup and configuration, as it is file-based and does not require an external database server.

> It is fast and efficient, making it a good choice for small to medium-scale ML projects.

> It is well-suited for single-user or small team environments, as it does not support concurrent access or multiple users writing to the same database at the same time.

To use SQLite as the tracking backend for MLflow, you simply need to set the MLflow tracking URI to the URI of the SQLite database file.

In [ ]:
!pip install mlflow --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00


We will import several Python packages and sets the MLflow tracking server URI to be a local SQLite database. It also suppresses any warnings and prints the version of MLflow that is installed.



In [ ]:
import os
import shutil
import pprint

from random import random, randint
import mlflow.sklearn
from mlflow import log_metric, log_param, log_artifacts
from sklearn.ensemble import RandomForestRegressor
from mlflow.tracking import MlflowClient
import warnings

In [ ]:
warnings.filterwarnings("ignore")
print(mlflow.__version__)

2.2.2


This cell loads the iris dataset, sets the MLflow tracking server URI to be a local SQLite database, trains a random forest classifier on the iris dataset, and logs the hyperparameters and accuracy metric to the MLflow tracking server. It also registers the trained model with the name "iris-rf" on the MLflow tracking server.


In [ ]:
import mlflow
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load iris dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# Set the tracking server to be localhost with sqlite as tracking store
mlflow.set_tracking_uri("sqlite:///mlruns.db")

# Train a random forest classifier and log parameters and metrics
with mlflow.start_run(run_name="iris-rf") as run:
    params = {"n_estimators": 100, "max_depth": 5}
    clf = RandomForestClassifier(**params)
    clf.fit(X_train, y_train)

    mlflow.log_params(params)
    mlflow.log_metric("accuracy", clf.score(X_test, y_test))

    # Log and register the model at the same time
    mlflow.sklearn.log_model(clf, "model")
    mlflow.sklearn.log_model(clf, "model", registered_model_name="iris-rf")


2023/04/12 11:25:23 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/12 11:25:23 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

Here we retrieve a list of all registered models from the MLflow tracking server and print their names and latest version numbers.

In [ ]:
import mlflow

client = mlflow.tracking.MlflowClient()

# Get a list of all registered models
model_names = set()
for mv in client.search_model_versions(""):
    model_name = mv.name
    model_names.add(model_name)

for model_name in model_names:
    latest_version = client.get_latest_versions(model_name, stages=None)[0]
    print(model_name, latest_version.version)


iris-rf 1


This cell starts the MLflow tracking UI server in the background on port 5000 with a SQLite database backend. 


In [ ]:
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --backend-store-uri sqlite:///mlruns.db --port 5000 &")# run tracking UI in the background

We then install the Pyngrok package and set up an HTTPS tunnel using Ngrok to allow access to the MLflow tracking UI from a remote browser.


After installing Pyngrok, the cell prompts the user to enter their Ngrok authtoken, which is required to create an HTTPS tunnel. Once the authtoken is set, it uses the ngrok.connect function to create the tunnel, and prints the public URL that can be used to access the MLflow tracking UI.



In [ ]:
# create remote tunnel using ngrok.com to allow local port access
# borrowed from https://colab.research.google.com/github/alfozan/MLflow-GBRT-demo/blob/master/MLflow-GBRT-demo.ipynb#scrollTo=4h3bKHMYUIG6
!pip install pyngrok --quiet
from pyngrok import ngrok
from getpass import getpass


# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

Enter the ngrok authtoken: ··········
MLflow Tracking UI: https://e9bc-34-150-178-14.ngrok-free.app


Finally, the cell connects to the SQLite database used by the MLflow tracking server and executes two SQL queries to retrieve the names of all tables and all parameter values, and prints the results. 



In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("/content/mlruns.db")

# Execute a query and print the results
cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table'")
for row in cursor:
    print(row[0])


experiments
alembic_version
experiment_tags
tags
registered_models
runs
registered_model_tags
model_version_tags
model_versions
latest_metrics
metrics
params


In [ ]:
# Execute a query and print the results
cursor = conn.execute("SELECT * FROM params")
for row in cursor:
    print(row)

('n_estimators', '100', '0626c9e2a35c43cf9932fdb3f135453c')
('max_depth', '5', '0626c9e2a35c43cf9932fdb3f135453c')


<img src="https://raw.githubusercontent.com/aaubs/ds-master/main/data/Images/Exercise.png" width="600">